In [49]:

import pandas as pd
import json
from pkg_resources import resource_filename
import numpy as np
from pprint import pprint

In [50]:
data_path=r'C:\Users\sebas\IDrive-Sync\MetaLocalLLC\RL-Bots\Tournament-gym\Data'
MW='M'
tournament=pd.read_csv(f'{data_path}/{MW}NCAATourneyCompactResults.csv')
seeds=pd.read_csv(f'{data_path}/{MW}NCAATourneySeeds.csv')
slots=pd.read_csv(f'{data_path}/{MW}NCAATourneySlots.csv')
teams=pd.read_csv(f'{data_path}/{MW}Teams.csv')
regions=pd.read_csv(f'{data_path}/{MW}Seasons.csv')
teams=teams[['TeamID','TeamName']]

In [51]:
seeds

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374
...,...,...,...
2549,2024,Z12,1241
2550,2024,Z13,1436
2551,2024,Z14,1324
2552,2024,Z15,1443


In [52]:
if MW=='M':
    add_game=tournament.loc[[tournament.index[0]]].copy()
    # in 2021 this game got forfitted cux covid
    # Add 'Oregon'
    add_game['Season']=2021
    # add_game['WTeamName']='Oregon'
    add_game['WTeamID']=1332
    # add_game['WSeed']='X07'
    add_game['WScore']=1
    # Add VCU
    add_game['LTeamID']=1433
    # add_game['WTeamName']='VCU'
    # add_game['Lseed']='X10'
    add_game['LScore']=0
    add_game['WLoc']='N'
    add_game['NumOT']=0

    tournament=pd.concat([tournament,add_game],axis=0,ignore_index=True)


In [53]:

seeds=pd.merge(seeds,teams,on='TeamID',how='left')
# seeds[seeds['TeamName']=='VCU']

In [54]:
test_season=2021
tourn=tournament[tournament['Season']==test_season]
tourn.head(1)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
2251,2021,136,1179,53,1455,52,N,0


In [55]:

def get_full_results(season):
    tourn=tournament[tournament['Season']==season]
    seed=seeds[seeds['Season']==season]

    left_tourn=pd.merge(tourn,seed,left_on='WTeamID',right_on='TeamID',how='left')
    left_tourn=left_tourn.rename(columns={'Seed':'WSeed','Season_x':"Season","TeamName":'WTeamName'})
    left_tourn=left_tourn.drop(['Season_y','TeamID'],axis=1)
    right_tourn=pd.merge(left_tourn,seed,left_on='LTeamID',right_on='TeamID',how='left')
    right_tourn=right_tourn.rename(columns={'Seed':'LSeed','Season_x':"Season","TeamName":'LTeamName'})
    
    tourn_results=right_tourn.drop(['Season_y','TeamID','WScore','LScore','WLoc','NumOT'],axis=1)
    tourn_results['Matchup']=[[x for x in c] for c in tourn_results[['WTeamName','LTeamName']].values]
    return tourn_results

def get_playins(tourn_results):
    Day1=tourn_results.sort_values('DayNum').groupby('WTeamID').head(1).sort_values('WSeed')
    playin_regex=r"[A-Z][0-9][0-9][a-z]"
    seed_a_mask=Day1['WSeed'].str.match(playin_regex)
    seed_b_mask=Day1['LSeed'].str.match(playin_regex)
    playin_ids=Day1[(seed_a_mask) & (seed_b_mask)].index
    play_ins=Day1.loc[playin_ids].copy()
    play_ins['next_slot']=play_ins['WSeed'].str.replace(r'[a-z]','',regex=True)
    return play_ins

def get_round_results(tourn_results,round):
    play_ins=get_playins(tourn_results)
    if round ==0:
        return play_ins
    else:
        round=round-1
        playin_ids=play_ins.index
        tourn_results=tourn_results.drop(playin_ids,axis=0)

    # print(round)
    round_results=tourn_results.sort_values('DayNum',ascending=True).groupby('WTeamID').nth(round).sort_values('WSeed')
    round_results['WSeed']=round_results['WSeed'].str.replace(r'[a-z]','',regex=True)
    return round_results

def get_slot_mapping(slots,season=2023):
    slot=slots[slots['Season']==season].reset_index(drop=True)
    playin_regex=r"[A-Z][0-9][0-9]"
    R0=slot[slot['Slot'].str.match(playin_regex)].copy()
    R1=slot[slot['Slot'].str.contains('R1')].copy()
    R2=slot[slot['Slot'].str.contains('R2')].copy()
    R3=slot[slot['Slot'].str.contains('R3')].copy()
    R4=slot[slot['Slot'].str.contains('R4')].copy()
    R5=slot[slot['Slot'].str.contains('R5')].copy()
    R6=slot[slot['Slot'].str.contains('R6')].copy()

    round_slots={'Round0':R0,'Round1':R1,'Round2':R2,'Round3':R3,'Round4':R4,'Round5':R5,'Round6':R6}
    round_maps={}
    for round,r_slots in round_slots.items():
        strong_map=r_slots.set_index('StrongSeed')['Slot'].to_dict()
        weak_map=r_slots.set_index('WeakSeed')['Slot'].to_dict()
        
        strong_map.update(weak_map)
        round_map={f'{round}_map':strong_map}
        round_maps[round]=round_map
    return round_maps

In [56]:
round_maps=get_slot_mapping(slots,season=2022)
round_maps

{'Round0': {'Round0_map': {'W12a': 'W12',
   'X11a': 'X11',
   'Y16a': 'Y16',
   'Z16a': 'Z16',
   'W12b': 'W12',
   'X11b': 'X11',
   'Y16b': 'Y16',
   'Z16b': 'Z16'}},
 'Round1': {'Round1_map': {'W01': 'R1W1',
   'W02': 'R1W2',
   'W03': 'R1W3',
   'W04': 'R1W4',
   'W05': 'R1W5',
   'W06': 'R1W6',
   'W07': 'R1W7',
   'W08': 'R1W8',
   'X01': 'R1X1',
   'X02': 'R1X2',
   'X03': 'R1X3',
   'X04': 'R1X4',
   'X05': 'R1X5',
   'X06': 'R1X6',
   'X07': 'R1X7',
   'X08': 'R1X8',
   'Y01': 'R1Y1',
   'Y02': 'R1Y2',
   'Y03': 'R1Y3',
   'Y04': 'R1Y4',
   'Y05': 'R1Y5',
   'Y06': 'R1Y6',
   'Y07': 'R1Y7',
   'Y08': 'R1Y8',
   'Z01': 'R1Z1',
   'Z02': 'R1Z2',
   'Z03': 'R1Z3',
   'Z04': 'R1Z4',
   'Z05': 'R1Z5',
   'Z06': 'R1Z6',
   'Z07': 'R1Z7',
   'Z08': 'R1Z8',
   'W16': 'R1W1',
   'W15': 'R1W2',
   'W14': 'R1W3',
   'W13': 'R1W4',
   'W12': 'R1W5',
   'W11': 'R1W6',
   'W10': 'R1W7',
   'W09': 'R1W8',
   'X16': 'R1X1',
   'X15': 'R1X2',
   'X14': 'R1X3',
   'X13': 'R1X4',
   'X12': 'R1X

In [57]:
test_season=2022
round=0
season_tournament=get_full_results(season=test_season)
round_results=get_round_results(season_tournament,round=round)
# type(round_results['Matchup'][0])
round_results

,Season,DayNum,WTeamID,LTeamID,WSeed,WTeamName,LSeed,LTeamName,Matchup,next_slot
0,2022,134,1231,1461,W12a,Indiana,W12b,Wyoming,"[Indiana, Wyoming]",W12
2,2022,135,1323,1353,X11a,Notre Dame,X11b,Rutgers,"[Notre Dame, Rutgers]",X11
1,2022,134,1411,1394,Y16b,TX Southern,Y16a,TAM C. Christi,"[TX Southern, TAM C. Christi]",Y16
3,2022,135,1460,1136,Z16b,Wright St,Z16a,Bryant,"[Wright St, Bryant]",Z16


In [58]:
def get_team_mapping(current_round):
    current_round['WSeed_n']=current_round['WSeed'].str.replace(r'[A-Za-z]','',regex=True).astype(int)
    current_round['WRegion']=current_round['WSeed'].str.replace(r'[0-9a-z]','',regex=True)
    current_round['LSeed_n']=current_round['LSeed'].str.replace(r'[A-Za-z]','',regex=True).astype(int)
    current_round['LRegion']=current_round['LSeed'].str.replace(r'[0-9a-z]','',regex=True)
    winner_map={'WRegion':'Region',
    'WSeed':'Seed',
    'WTeamName':'Team',
    }
    winners=current_round[[c for c in winner_map.keys()]].rename(columns=winner_map)

    loser_map={'LRegion':'Region',
    'LSeed':'Seed',
    'LTeamName':'Team',
    }
    losers=current_round[[c for c in loser_map.keys()]].rename(columns=loser_map)

    stacked=pd.concat([winners,losers])
    stacked=stacked.sort_values(['Seed','Region']).reset_index(drop=True)
    team_map=stacked.set_index('Team')['Seed'].to_dict()
    return team_map

In [59]:

def format_regions(current_round,round_num,final_dict):

    result_names={0:'playin',1:'first',2:'second',3:'sweet16', 4:'elite8', 5:'final4', 6:'championship'}
    current_round['WSeed_n']=current_round['WSeed'].str.replace(r'[A-Za-z]','',regex=True).astype(int)
    current_round['WRegion']=current_round['WSeed'].str.replace(r'[0-9a-z]','',regex=True)
    current_round['LSeed_n']=current_round['LSeed'].str.replace(r'[A-Za-z]','',regex=True).astype(int)
    current_round['LRegion']=current_round['LSeed'].str.replace(r'[0-9a-z]','',regex=True)

    winner_map={'WRegion':'Region',
    'WSeed_n':'Seed',
    'WTeamName':'Team',
    }
    winners=current_round[[c for c in winner_map.keys()]].rename(columns=winner_map)

    loser_map={'LRegion':'Region',
    'LSeed_n':'Seed',
    'LTeamName':'Team',
    }
    losers=current_round[[c for c in loser_map.keys()]].rename(columns=loser_map)

    stacked=pd.concat([winners,losers])
    stacked=stacked.sort_values(['Seed','Region']).reset_index(drop=True)
    region_dict={}
    for region,data in stacked.groupby('Region'):
        region_dict[region]=data[['Seed','Team']].to_dict('records')

    stacked=stacked.sort_values(['Seed','Team'])
    results=stacked[['Seed','Team']].to_dict('records')
    round_name=result_names[round_num]
    final_dict['Results'][round_name]=results
    if round_num==1:
        final_dict['Region']=region_dict
    if round_num==6:
        final_dict['Results']['winner']=winners[['Seed','Team']].to_dict('records')

    return final_dict

In [60]:
def format_tournament_results(tourny,season):
    rounds=[0,1,2,3,4,5,6]
    tournament_results={}
    bracket_shape={}
    json_formatted={}
    json_formatted['Results']={}

    round_maps = get_slot_mapping(slots,season=season)
    tournament_results['structure']={}
    tournament_results['results']={}
    tournament_results['matchups']={}
    for round in rounds:

        current_round = get_round_results(tourny,round=round)
        
        name=f'Round{round}'
        slot_map=round_maps[name]
        tournament_results['structure'][round]=slot_map[f'{name}_map']
        # print(f'{name}-------------------------------')
        
        if round==0:
            play_in_results=current_round.set_index('next_slot')['WTeamName'].to_dict()
            match_up=current_round.set_index('next_slot')['Matchup'].to_dict()


            tournament_results['results'][0]=play_in_results
            tournament_results['matchups'][0]=match_up

            
            # display(current_round)
            bracket_shape[round]=current_round.shape[0]
            continue
        

        elif round==1:
            current_round['current_slot']=current_round['WSeed'].copy()
            current_round['next_slot']=np.nan
            team_map= get_team_mapping(current_round)
            tournament_results['teams']=team_map


        else:
            current_round['current_slot']=current_round['WTeamName'].map(previous_map)
            current_round['next_slot']=np.nan

        for mapping in slot_map.values():
            # pprint(mapping)
            current_round['next_slot']=current_round['next_slot'].fillna(current_round['current_slot'].map(mapping))

        bracket_shape[round]=current_round.shape[0]

        previous_map=current_round.set_index('WTeamName')['next_slot'].to_dict()
        result_map=current_round.set_index('next_slot')['WTeamName'].to_dict()
        match_up=current_round.set_index('next_slot')['Matchup'].to_dict()
        
        tournament_results['results'][round]=result_map
        tournament_results[f'matchups'][round]=match_up
        # previous_map=result_map

        current_round=current_round.reset_index(drop=True)
        current_round.index=[i+1 for i in current_round.index]
        # display(current_round)
        if round==6:
            tournament_results['results'][7]=[current_round.loc[1,'WTeamName']]
            winning_name=[current_round.loc[1,'WTeamName']]
            print('WINNER',winning_name)

        json_formatted=format_regions(current_round,round_num=round,final_dict=json_formatted)
    print(season,bracket_shape)
    return tournament_results,json_formatted



# pprint(tournament_results)
# round_maps


In [61]:
test_season=2022
season_tournament=get_full_results(season=test_season)
season_results,json_formatted=format_tournament_results(season_tournament,test_season)


WINNER ['Kansas']
2022 {0: 4, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}


In [62]:
test_season=2024
slotmap24=get_slot_mapping(slots,season=2024)

res24={}
res24['structure']=slotmap24


In [63]:
seasons=slots.Season.unique()
ALL_RESULTS={}
FORMATTED_RESULTS={}
for season in seasons:
    if season not in [2020,2024]:
        
        season_tournament=get_full_results(season=season)
        season_results,json_formatted=format_tournament_results(season_tournament,season)
        ALL_RESULTS[str(season)]=season_results
        FORMATTED_RESULTS[str(season)]=json_formatted
ALL_RESULTS['2024']=res24


WINNER ['Villanova']
1985 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Louisville']
1986 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Indiana']
1987 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Kansas']
1988 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Michigan']
1989 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['UNLV']
1990 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Duke']
1991 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Duke']
1992 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['North Carolina']
1993 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Arkansas']
1994 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['UCLA']
1995 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Kentucky']
1996 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Arizona']
1997 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Kentucky']
1998 {0: 0, 1: 32, 2: 16, 3: 8, 4: 4, 5: 2, 6: 1}
WINNER ['Connecticut']
1999 {0:

In [64]:
ALL_RESULTS['2024']['structure']={n:struct[f'Round{n}_map'] for n,struct in enumerate(res24['structure'].values())}

In [65]:
ALL_RESULTS['2024']['structure']

{0: {'X16a': 'X16',
  'Y10a': 'Y10',
  'Y16a': 'Y16',
  'Z10a': 'Z10',
  'X16b': 'X16',
  'Y10b': 'Y10',
  'Y16b': 'Y16',
  'Z10b': 'Z10'},
 1: {'W01': 'R1W1',
  'W02': 'R1W2',
  'W03': 'R1W3',
  'W04': 'R1W4',
  'W05': 'R1W5',
  'W06': 'R1W6',
  'W07': 'R1W7',
  'W08': 'R1W8',
  'X01': 'R1X1',
  'X02': 'R1X2',
  'X03': 'R1X3',
  'X04': 'R1X4',
  'X05': 'R1X5',
  'X06': 'R1X6',
  'X07': 'R1X7',
  'X08': 'R1X8',
  'Y01': 'R1Y1',
  'Y02': 'R1Y2',
  'Y03': 'R1Y3',
  'Y04': 'R1Y4',
  'Y05': 'R1Y5',
  'Y06': 'R1Y6',
  'Y07': 'R1Y7',
  'Y08': 'R1Y8',
  'Z01': 'R1Z1',
  'Z02': 'R1Z2',
  'Z03': 'R1Z3',
  'Z04': 'R1Z4',
  'Z05': 'R1Z5',
  'Z06': 'R1Z6',
  'Z07': 'R1Z7',
  'Z08': 'R1Z8',
  'W16': 'R1W1',
  'W15': 'R1W2',
  'W14': 'R1W3',
  'W13': 'R1W4',
  'W12': 'R1W5',
  'W11': 'R1W6',
  'W10': 'R1W7',
  'W09': 'R1W8',
  'X16': 'R1X1',
  'X15': 'R1X2',
  'X14': 'R1X3',
  'X13': 'R1X4',
  'X12': 'R1X5',
  'X11': 'R1X6',
  'X10': 'R1X7',
  'X09': 'R1X8',
  'Y16': 'R1Y1',
  'Y15': 'R1Y2',
  'Y14'

In [66]:
ALL_RESULTS['2023']['structure']

{0: {'W16a': 'W16',
  'X16a': 'X16',
  'Y11a': 'Y11',
  'Z11a': 'Z11',
  'W16b': 'W16',
  'X16b': 'X16',
  'Y11b': 'Y11',
  'Z11b': 'Z11'},
 1: {'W01': 'R1W1',
  'W02': 'R1W2',
  'W03': 'R1W3',
  'W04': 'R1W4',
  'W05': 'R1W5',
  'W06': 'R1W6',
  'W07': 'R1W7',
  'W08': 'R1W8',
  'X01': 'R1X1',
  'X02': 'R1X2',
  'X03': 'R1X3',
  'X04': 'R1X4',
  'X05': 'R1X5',
  'X06': 'R1X6',
  'X07': 'R1X7',
  'X08': 'R1X8',
  'Y01': 'R1Y1',
  'Y02': 'R1Y2',
  'Y03': 'R1Y3',
  'Y04': 'R1Y4',
  'Y05': 'R1Y5',
  'Y06': 'R1Y6',
  'Y07': 'R1Y7',
  'Y08': 'R1Y8',
  'Z01': 'R1Z1',
  'Z02': 'R1Z2',
  'Z03': 'R1Z3',
  'Z04': 'R1Z4',
  'Z05': 'R1Z5',
  'Z06': 'R1Z6',
  'Z07': 'R1Z7',
  'Z08': 'R1Z8',
  'W16': 'R1W1',
  'W15': 'R1W2',
  'W14': 'R1W3',
  'W13': 'R1W4',
  'W12': 'R1W5',
  'W11': 'R1W6',
  'W10': 'R1W7',
  'W09': 'R1W8',
  'X16': 'R1X1',
  'X15': 'R1X2',
  'X14': 'R1X3',
  'X13': 'R1X4',
  'X12': 'R1X5',
  'X11': 'R1X6',
  'X10': 'R1X7',
  'X09': 'R1X8',
  'Y16': 'R1Y1',
  'Y15': 'R1Y2',
  'Y14'

In [67]:
import os
export_path=r'C:\Users\sebas\IDrive-Sync\MetaLocalLLC\RL-Bots\Tournament-gym\bracketology_rl\data'
export_path=os.path.join(export_path,f'{MW}better_brackets2024.json')

In [68]:
import json


with open(export_path , 'w') as f:
    json.dump(ALL_RESULTS, f)

In [69]:
ALL_RESULTS['2024']

{'structure': {0: {'X16a': 'X16',
   'Y10a': 'Y10',
   'Y16a': 'Y16',
   'Z10a': 'Z10',
   'X16b': 'X16',
   'Y10b': 'Y10',
   'Y16b': 'Y16',
   'Z10b': 'Z10'},
  1: {'W01': 'R1W1',
   'W02': 'R1W2',
   'W03': 'R1W3',
   'W04': 'R1W4',
   'W05': 'R1W5',
   'W06': 'R1W6',
   'W07': 'R1W7',
   'W08': 'R1W8',
   'X01': 'R1X1',
   'X02': 'R1X2',
   'X03': 'R1X3',
   'X04': 'R1X4',
   'X05': 'R1X5',
   'X06': 'R1X6',
   'X07': 'R1X7',
   'X08': 'R1X8',
   'Y01': 'R1Y1',
   'Y02': 'R1Y2',
   'Y03': 'R1Y3',
   'Y04': 'R1Y4',
   'Y05': 'R1Y5',
   'Y06': 'R1Y6',
   'Y07': 'R1Y7',
   'Y08': 'R1Y8',
   'Z01': 'R1Z1',
   'Z02': 'R1Z2',
   'Z03': 'R1Z3',
   'Z04': 'R1Z4',
   'Z05': 'R1Z5',
   'Z06': 'R1Z6',
   'Z07': 'R1Z7',
   'Z08': 'R1Z8',
   'W16': 'R1W1',
   'W15': 'R1W2',
   'W14': 'R1W3',
   'W13': 'R1W4',
   'W12': 'R1W5',
   'W11': 'R1W6',
   'W10': 'R1W7',
   'W09': 'R1W8',
   'X16': 'R1X1',
   'X15': 'R1X2',
   'X14': 'R1X3',
   'X13': 'R1X4',
   'X12': 'R1X5',
   'X11': 'R1X6',
   'X10'

In [70]:

import json
export_base_path=r'C:\Users\sebas\IDrive-Sync\MetaLocalLLC\RL-Bots\Tournament-gym\bracketology_rl\data'
old_export_path=os.path.join(export_base_path,f'{MW}old_brackets2024.json')
with open(old_export_path, 'w') as f:
    json.dump(FORMATTED_RESULTS, f)